In [1]:
%load_ext autoreload
%autoreload 2

In [68]:
from lib.scrape import fetch_urls
from lib.write import write_json_lines_partition
import pandas as pd
import re
import json

In [29]:
csv_file_path = "../data/helavault_aug2023.csv"
collection_df = pd.read_csv("../data/operational/collection.csv")

In [60]:
card_details_json, failures = fetch_urls(list(set(collection_df['url'])))

Fetching URLs: 100%|██████████| 2242/2242 [10:14<00:00,  3.65URL/s, Est. Time Remaining=0.00s]    


In [61]:
failures

[('https://json.edhrec.com/pages/cards/a-dragons-rage-channeler.json',
  requests.exceptions.HTTPError('403 Client Error: Forbidden for url: https://json.edhrec.com/pages/cards/a-dragons-rage-channeler.json')),
 ('https://json.edhrec.com/pages/cards/sance.json',
  requests.exceptions.HTTPError('403 Client Error: Forbidden for url: https://json.edhrec.com/pages/cards/sance.json'))]

In [62]:
# TODO use yielding fetch_urls
card_details_json_fixed, failures_fixed = fetch_urls(
    ['https://json.edhrec.com/pages/cards/seance.json', 'https://json.edhrec.com/pages/cards/dragons-rage-channeler.json']
)

Fetching URLs: 100%|██████████| 2/2 [00:00<00:00, 12.82URL/s, Est. Time Remaining=0.00s]


In [64]:
card_details_json += card_details_json_fixed

In [71]:
json_file_directory = "../data/staging/card_details"
write_json_lines_partition(card_details_json, 64, json_file_directory)

In [88]:
def get_commanders(card_detail):
    if "cardlists" not in card_detail["container"]["json_dict"]:
        return []
    return next(
        (
            item for item in card_detail["container"]["json_dict"]["cardlists"]
            if item["header"] == "Top Commanders"
        ), 
        {"cardviews": []}
    )["cardviews"]

operational_card_details = [
    {
        **card_detail["container"]["json_dict"]["card"],
        "commanders": get_commanders(card_detail)
    }
    for card_detail in card_details_json
    if "container" in card_detail
]

In [91]:
json_file_directory = "../data/operational/card_details"
write_json_lines_partition(operational_card_details, 64, json_file_directory)